In [ ]:
import sys, traceback
import cv2
import numpy as np
import argparse
import string
from plantcv import plantcv as pcv

In [ ]:
class options:
    def __init__(self):
        self.image = "img/tutorial_images/vis/original_image.jpg"
        self.debug = "plot"
        self.writeimg= False 
        self.result = "./vis_tutorial_results"
        self.outdir = "."
# Get options
args = options()

# Set debug to the global parameter 
pcv.params.debug = args.debug

In [ ]:
# Read image
img, path, filename = pcv.readimage(args.image)

In [ ]:
# Convert RGB to HSV and extract the saturation channel
s = pcv.rgb2gray_hsv(img, 's')

In [ ]:
# Threshold the Saturation image
s_thresh = pcv.binary_threshold(s, 30, 255, 'light')

In [ ]:
# Median Blur
s_mblur = pcv.median_blur(s_thresh, 5)
s_cnt = pcv.median_blur(s_thresh, 5)

In [ ]:
# Convert RGB to LAB and extract the blue channel
b = pcv.rgb2gray_lab(img, 'b')

# Threshold the blue image
b_thresh = pcv.threshold.binary(b, 129, 255, 'light')
b_cnt = pcv.threshold.binary(b, 19, 255, 'light')

In [ ]:
# Join the thresholded saturation and blue-yellow images
bs = pcv.logical_and(s_mblur, b_cnt)

In [ ]:
# Apply Mask (for VIS images, mask_color=white)
masked = pcv.apply_mask(img, bs, 'white')

In [ ]:
# Identify objects
id_objects,obj_hierarchy = pcv.find_objects(masked, bs)

In [ ]:
# Define ROI
roi1, roi_hierarchy = pcv.roi.rectangle(600,450,-600,-700, img)

In [ ]:
# Decide which objects to keep
roi_objects, hierarchy, kept_mask, obj_area = pcv.roi_objects(img,'partial',roi1,roi_hierarchy,id_objects,obj_hierarchy)

In [ ]:
# Object combine kept objects
obj, mask = pcv.object_composition(img, roi_objects, hierarchy)

In [ ]:
############### Analysis ################  
  
# Find shape properties, output shape image (optional)
shape_header, shape_data, shape_img = pcv.analyze_object(img, obj, mask, args.outdir + '/' + filename)

# Shape properties relative to user boundary line (optional)
boundary_header, boundary_data, boundary_img1 = pcv.analyze_bound_horizontal(img, obj, mask, 1680, args.outdir + '/' + filename)

# Determine color properties: Histograms, Color Slices and Pseudocolored Images, output color analyzed images (optional)
color_header, color_data, color_img = pcv.analyze_color(img, kept_mask, 256, 'all', 'rgb', 'v', 'img', 300, args.outdir + '/' + filename)

# Write shape and color data to results file
result=open(args.result,"a")
result.write('\t'.join(map(str,shape_header)))
result.write("\n")
result.write('\t'.join(map(str,shape_data)))
result.write("\n")
result.close()